In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from random import shuffle
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

Lets import the data

In [2]:
data = pd.read_csv("../Clean_Data/Clean_CDC.csv")
data.Date = pd.to_datetime(data["Date"])
data.State = data.State.replace('NYC', 'NY')
NYData = data[data.State == 'NY'].groupby('Date', as_index=False).sum()
data = data[data.State != 'NY']
data = pd.concat((data, NYData)).replace(np.nan, 'NY')
data_usa = data.groupby('Date').sum().reset_index()

In [3]:
data.head()

,Date,State,Total_cases,Confirmed_cases,New_cases,Total_deaths,Confirmed_death,New_deaths
0,2020-01-22,CO,0,0,0,0,0,0
1,2020-01-23,CO,0,0,0,0,0,0
2,2020-01-24,CO,0,0,0,0,0,0
3,2020-01-25,CO,0,0,0,0,0,0
4,2020-01-26,CO,0,0,0,0,0,0


Functions used in this notebook.

In [4]:
def package_data(data, scaler, states=["CA"], category="Total_cases", window=7):
    """Function prepares data to be trained by the RNN
    args:
        data: Pandas dataframe of all data
        scaler: scikit Scaler. Scales data
        state: string. The state we want to train on
        category: String. The data field of interest
        window: int. size of window to use for independent varaible X.
            Default is weekly
    Returns:
        training_data, testing_data: vectors that contain training data [X, y]
    """
    stateData = data[data["State"].isin(states)].pivot(
        index='Date', columns='State', values=[category])
    stateData.New_deaths = scaler.fit_transform(stateData.New_deaths)
    Data = [(stateData.New_deaths.values[i:i+window],
             stateData.New_deaths.values[i+window+1]) for i in range(0,291-window-1,1)]
    shuffle(Data)
    split = int(len(Data)*.7)
    training_data = np.array(Data[:split])
    testing_data = np.array(Data[split+1:])
    train_x = np.hstack(training_data[:,0])
    train_y = np.vstack(training_data[:,1])
    test_x = np.hstack(testing_data[:,0])
    test_y = np.hstack(testing_data[:,1])
    return stateData.New_deaths, [train_x,train_y], [test_x,test_y]

def package_data2(data, scaler, state, category, window=7):
    """Function prepares data to be trained by the RNN
    args:
        data: Pandas dataframe of all data
        scaler: scikit Scaler. Scales data
        state: string. The state we want to train on
        category: String. The data field of interest
        window: int. size of window to use for independent varaible X.
            Default is weekly
    Returns:
        training_data, testing_data: vectors that contain training data [X, y]
    """
    stateData = data[data["State"] == state].pivot(
        index='Date', columns='State', values=[category])
    stateData[category] = scaler.fit_transform(stateData[category])
    Data = [(stateData[category].values[i:i+window],
             stateData[category].values[i+window+1]) for i in range(0,291-window-1,1)]
    split = int(len(Data)*.7)
    training_data = np.array(Data[:split])
    testing_data = np.array(Data[split+1:])
    train_x = np.expand_dims(np.hstack(training_data[:,0]).T,2)
    train_y = np.vstack(training_data[:,1])
    test_x = np.expand_dims(np.hstack(testing_data[:,0]).T,2)
    test_y = np.hstack(testing_data[:,1])
    return [train_x,train_y],[test_x,test_y]

def make_forecast(model, days, start_x, window=7):
    """Function makes predictions given the model
    model: Tensorflow Model. Trainined tensorflow model
    days: number of days to perform the forecast for
    start_x: numpy array. Starting date for forecast
    """
    predicted = [i for i in start_x.tolist()[0]]
    for i in range(0,days):
        x = np.expand_dims(np.vstack(predicted[-1*window:]),0)
        new_y = model.predict(x)
        predicted.append(new_y[0].tolist())
    return predicted

def build_network(input_shape, units):
    """
    Function builds an lstm network and returns it
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=input_shape))
    model.add(tf.keras.layers.LSTM(units, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.LSTM(units, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.LSTM(units))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(units, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(25, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1))
    return model

The below function is used to find the mean squared error for any state and category.

In [5]:
def LSTM_traim(data, states=["CA"], categories=["Total_cases"], window=7,
              units = 50):
    """
    Function trains an RNN to perform forcasting
    args:
        data: Pandas DataFrame of data.
        states: List of strings. States to perform analysis for
        category: List of strings. Categories to perform analysis for
        window: int. Size of window to use for independent variable x
        units: int. Number of recurrent units in network
    returns:
        mae: Mean squared error of model
    """
    scaler = MinMaxScaler(feature_range=(0,1))
    result = []
    for state in states:
        for category in categories:
            cur_result = {}
            train_data,test_data = package_data2(data, scaler, state, category, window)
            model = build_network((window, 1), units)
            model.compile(
                loss = tf.keras.losses.MeanSquaredError(),
                optimizer = tf.keras.optimizers.SGD(),
                metrics=["mse"],
            )
            X = train_data[0]
            y = train_data[1]
            model.fit([X], [y], 
                      validation_split=0.7,
                      epochs=200, 
                      batch_size=1, 
                      verbose=0)
            cur_result["State"] = state
            cur_result["Category"] = category
            mresult = model.evaluate(test_data[0],test_data[1])
            cur_result["mse"] = scaler.inverse_transform(
                np.reshape(mresult[0], (-1,1)))
            result.append(cur_result)
    return result

In [6]:
#LSTM_traim(data, states=["CA", "NY"], 
#           categories=["New_deaths"], window=7, units = 50)

# Forecasting Categories for USA

In [7]:
data_usa_total_cases = data_usa.Total_cases
data_usa_new_cases = data_usa.New_cases
data_usa_new_deaths = data_usa.New_deaths

## Forecast USA New Cases using LSTM

In [18]:
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(np.reshape(data_usa_new_cases.values, (-1,1)))
window = 14
X = [(X[i:i+window],
         X[i+window+1]) for i in range(0,291-window-1,1)]

split = int(len(X)*.7)
training_data = np.array(X[:split])
testing_data = np.array(X[split+1:])
train_x = np.expand_dims(np.hstack(training_data[:,0]).T,2)
train_y = np.vstack(training_data[:,1])
test_x = np.expand_dims(np.hstack(testing_data[:,0]).T,2)
test_y = np.hstack(testing_data[:,1])

In [19]:
model = build_network((window, 1), 50)
model.summary()
model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics=["mse"],
)
model.fit(train_x, 
          train_y, 
          validation_split=0,
          epochs=200,
          batch_size=1, 
          verbose=0)
results = model.evaluate(test_x,test_y)
print("MSE:", scaler.inverse_transform(np.reshape(results[0], (-1,1))))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 14, 50)            10400     
_________________________________________________________________
dropout_12 (Dropout)         (None, 14, 50)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 14, 50)            20200     
_________________________________________________________________
dropout_13 (Dropout)         (None, 14, 50)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_14 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)               

The means squared error here is approximately 2988.

In [20]:
last_window = [i[0] for i in X[-1*window:]]
last_window = np.array(last_window)
pred = make_forecast(model, 30, last_window, window)
pred = scaler.inverse_transform(pred)[window:]

In [21]:
fig = make_subplots()
fig.add_trace(
    go.Scatter(y=data_usa_new_cases.values, name="New Cases"),
)

fig.add_trace(
    go.Scatter(x=[
        i for i in range(
            len(data_usa_new_cases.values), 
           len(data_usa_new_cases.values)+len(pred))],y=pred.T.tolist()[0], name="Forecast",
               #mode='lines',
              line=dict(color='rgb(0,100,80)')),
)
fig.update_layout(
    title_text="Forecast US New Covid-19 Cases for next 30 Days"
)
fig.update_xaxes(title_text="Days")
fig.update_yaxes(title_text="Cases")
fig.show()

## Forecast USA New Deaths using LSTM

In [12]:
scaler = MinMaxScaler(feature_range=(0,1))
X = scaler.fit_transform(np.reshape(data_usa_new_deaths.values, (-1,1)))
window = 14
X = [(X[i:i+window],
         X[i+window+1]) for i in range(0,291-window-1,1)]

split = int(len(X)*.7)
training_data = np.array(X[:split])
testing_data = np.array(X[split+1:])
train_x = np.expand_dims(np.hstack(training_data[:,0]).T,2)
train_y = np.vstack(training_data[:,1])
test_x = np.expand_dims(np.hstack(testing_data[:,0]).T,2)
test_y = np.hstack(testing_data[:,1])

In [13]:
model = build_network((window, 1), 50)
model.summary()
model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(),
    metrics=["mse"],
)
model.fit(train_x, 
          train_y, 
          validation_split=0,
          epochs=200,
          batch_size=1, 
          verbose=0)
results = model.evaluate(test_x,test_y)
print("MSE:", scaler.inverse_transform(np.reshape(results[0], (-1,1))))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 14, 50)            10400     
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 50)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 14, 50)            20200     
_________________________________________________________________
dropout_7 (Dropout)          (None, 14, 50)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_8 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)               

The means squared error here is approximately 2988.

In [14]:
last_window = [i[0] for i in X[-1*window:]]
last_window = np.array(last_window)
pred = make_forecast(model, 30, last_window, window)
pred = scaler.inverse_transform(pred)[window:]

In [15]:
fig = make_subplots()
fig.add_trace(
    go.Scatter(y=data_usa_new_deaths.values, name="New Deaths"),
)

fig.add_trace(
    go.Scatter(x=[
        i for i in range(
            len(data_usa_new_deaths.values), 
           len(data_usa_new_deaths.values)+len(pred))],y=pred.T.tolist()[0], name="Forecast",
               #mode='lines',
              line=dict(color='rgb(0,100,80)')),
)
fig.update_layout(
    title_text="Forecast US New Covid-19 Cases for next 30 Days"
)
fig.update_xaxes(title_text="Days")
fig.update_yaxes(title_text="Cases")
fig.show()

# Conclusion

Forecasting with LSTM's is quite difficult with the limited amount of data that we have here. Given this fact, the performance of the LSTM for large values (Total cases) was quite bad. Furthermore, it is also difficult to capture the sequential patterns in the data. Although this is an interesting case study, I would stick to ARIMA models.